# 순환 신경망(Recurrent neural networks)

이전 모듈에서는 텍스트의 풍부한 의미 표현과 임베딩 위에 간단한 선형 분류기를 사용했다. 이 아키텍처가 하는 일은 문장에서 단어의 의미를 집계하여 포착하는 것이지만, 임베딩 다음에 있는 집계 작업이 원래 텍스트에서 이 정보를 제거했기 때문에 단어의 **순서(order)** 를 고려하지 않는다. 이러한 모델들은 단어 순서를 모델링할 수 없기 때문에 텍스트 생성 또는 질문 답변과 같은 더 복잡하거나 모호한 작업을 해결할 수 없다.

텍스트 시퀀스의 의미를 포착하려면 **refurrent neural network** 또는 RNN이라고 불리는 다른 신경망 아키텍처를 사용해야 한다. RNN에서 우리는 한 번에 하나의 기호(symbol)를 네트워크를 통해 문장을 전달하며, 네트워크에서 일부 **state**를 생성하여 그 다음 기호를 사용하여 네트워크에 다시 전달한다.

![Image showing an example recurrent neural network generation.](./images/sample-rnn-model-generation.png)

토큰의 시퀀스 $X_0,\dots,X_n$가 주어지면 RNN은 신경망 블록의 시퀀스를 생성하고 역전파를 사용하여 이 시퀀스를 end-to-end 훈련시킨다. 각 네트워크 블록은 $(X_i,S_i)$ 쌍을 입력으로 가져와서 $S_{i+1}$를 생성한다. 최종 상태 $S_n$ 또는 출력 $X_n$은 결과를 생성하기 위해 선형 분류기로 들어간다. 모든 네트워크 블록은 동일한 가중치를 공유하며, 하나의 역전파 패스를 사용하여 end-to-end 훈련 얻는다.

상태 벡터 $S_0,\dots,S_n$은 네트워크를 통해 전달되기 때문에 단어 사이의 순차적 종속성(sequential dependencies)을 학습할 수 있다. 예를 들어, *not*라는 단어가 시퀀스 어딘가에 나타날 때, 상태 벡터 내의 특정 원소를 부정하는 방법을 배울 수 있으며, 결과적으로 부정이 발생한다.

순환 신경망이 뉴스 데이터셋을 분류하는 데 어떻게 도움이 되는지 살펴봅시다.

In [1]:
import torch
import torchtext
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)

Loading dataset...
Building vocab...


## 간단한 RNN 분류기

단순한 RNN의 경우, 각 순환 단위는 연결된 입력 벡터(input vector)와 상태 벡터(state vector)를 사용하고 새로운 상태 벡터를 생성하는 단순한 선형 네트워크이다. PyTorch는 이 유닛을 `RNNCell` 클래스로 나타내며, `RNN` 레이어와 같은 셀의 네트워크를 나타낸다.

RNN 분류기를 정의하기에 앞서 먼저 입력 사전(vocabulary)의 차원 수를 낮추기 위해 임베딩 레이어를 적용한 다음 그 다음에 RNN 레이어를 둘 것이다:

In [2]:
class RNNClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.rnn = torch.nn.RNN(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,h = self.rnn(x)
        return self.fc(x.mean(dim=1))

> **참조:** 여기서는 단순성을 위해 훈련되지 않은 임베딩 레이어를 사용하지만, 더 나은 결과를 위해 이전 섹션에서 설명한 Word2Vec 또는 GloVe 임베딩과 함께 사전 훈련된 임베딩 레이어를 사용할 수 있다. 더 잘 이해하기 위해 사전 훈련된 임베딩으로 작동하도록 이 코드를 조정할 수 있다.

우리의 경우 패딩된 데이터로더를 사용할 것이기 때문에 각 배치에는 동일한 길이의 패딩된 시퀀스가 여러 개 있을 것이다. RNN 레이어는 임베딩 텐서의 시퀀스를 입력으로 받고 다음과 같은 2가지 출력을 생성한다:
* $x$ 는 각 스텝의 RNN 셀 출력의 시퀀스이다.
* $h$ 는 시퀀스의 마지막 원소를 위한 마지막 은닉 층이다.

그런 다음 fully-connected 선형 분류기를 적용하여 클래스 수를 얻는다.

> **참조:** RNN 셀이 시퀀스 길이를 따라 unroll되면 역전파에 관련된 결과 레이어 수가 상당히 크기 때문에 RNN은 훈련시키기가 상당히 어렵다. 따라서 우리는 작은 learning rate를 선택하고, 좋은 결과를 내기 위해 더 큰 데이터셋에 대한 네트워크를 훈련시킬 필요가 있다. 시간이 꽤 걸릴 수 있기 때문에 GPU를 사용하는 것이 좋습니다.

In [3]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)
net = RNNClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.30375
6400: acc=0.38125
9600: acc=0.43854166666666666
12800: acc=0.47828125
16000: acc=0.5113125
19200: acc=0.5386458333333334
22400: acc=0.5616517857142858
25600: acc=0.5828125
28800: acc=0.6015972222222222
32000: acc=0.61915625
35200: acc=0.6344886363636364
38400: acc=0.6489322916666667
41600: acc=0.661826923076923
44800: acc=0.6726339285714286
48000: acc=0.6828333333333333
51200: acc=0.69189453125
54400: acc=0.7007169117647059
57600: acc=0.7082118055555555
60800: acc=0.715608552631579
64000: acc=0.722375
67200: acc=0.7286458333333333
70400: acc=0.7340482954545454
73600: acc=0.7396467391304348
76800: acc=0.7448177083333334
80000: acc=0.749625
83200: acc=0.7540384615384615
86400: acc=0.7585995370370371
89600: acc=0.7626004464285714
92800: acc=0.7659806034482759
96000: acc=0.7694791666666667
99200: acc=0.7731653225806452
102400: acc=0.77615234375
105600: acc=0.7791950757575757
108800: acc=0.7823253676470588
112000: acc=0.7850892857142857
115200: acc=0.7879774305555556
11840

(0.03507950032552083, 0.7917)

## Long Short Term Memory (LSTM)

고전적인 RNN의 주요 문제 중 하나는 소위 **기울기 소실(vanishing gradients)** 문제이다. RNN은 역전파 패스에서 end-to-end 훈련을 받기 때문에 네트워크의 첫 번째 레이어로 오류를 전파하는 데 어려움을 겪으며, 따라서 네트워크는 원거리 토큰 간의 관계를 학습할 수 없다. 이 문제를 피하는 방법 중 하나는 소위  **게이트(gates)** 를 사용하여 **explicit state management**를 도입하는 것이다. 이러한 종류의 가장 잘 알려진 두 가지 아키텍처가 있다: **Long Short Term Memory** (LSTM)과 **Gated Relay Unit** (GRU)이다.

![Image showing an example long short term memory cell](./images/long-short-term-memory-cell.svg)

LSTM 네트워크는 RNN과 유사한 방식으로 구성되지만 레이어 간에 전달되고 있는 두 가지 상태, 즉 actual state $c$와 hidden vector $h$가 있다. 각 단위에서, hidden vector $h_i$는 입력 $x_i$와 연결되며, 그들은 **게이트(gates)** 를 통해 actual state $c$에 발생하는 것을 제어한다. 각 게이트는 시그모이드 활성화 함수($0,1$ 범위의 출력)를 가진 신경망으로, 상태 벡터에 곱할 때 bitwise mask로 생각할 수 있다.

* **forget gate** 는 숨겨진 벡터를 가져와서 벡터 $c$의 어떤 성분을 우리가 잊어야 할지 통과해야 할지를 결정한다.
* **input gate** 는 입력 및 은닉 벡터로부터 일부 정보를 가져와 상태로 삽입합니다.
* **output gate** 는 $\tanh$ 활성화 함수로 일부 선형 레이어을 통해 상태를 변환한 다음 숨겨진 벡터 $h_i$를 사용하여 일부 컴포넌트를 선택하여 새로운 상태 $c_{i+1}$를 생성한다.

상태 $c$의 컴포넌트는 켜거나 끌 수 있는 플래그로 생각할 수 있다. 예를 들어, 우리가 시퀀스에서 *Alice*라는 이름을 만났을 때, 우리는 그것이 여성 캐릭터를 지칭한다고 가정하고, 문장에 여성 명사가 있는 상태에서 플래그를 올리고 싶을 수도 있다. 반면에 *Tom* 이라는 문구가 더 나오면 복수 명사라는 깃발을 올리겠습니다. 따라서 우리는 상태를 조작함으로써 문장 부분의 문법적 특성을 추적할 수 있을 것으로 추정된다.

> **참조**: LSTM 내부 구조를 이해하기 위한 훌륭한 자료인 Christopher Olah의 사이트를 참조 [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)

LSTM 셀의 내부 구조가 복잡해 보일 수 있지만, PyTorch는 `LSTMCell` 클래스 안에 이 구현을 숨기고 `LSTM` 객체를 제공하여 LSTM 계층을 전체적으로 나타낸다. 따라서 LSTM 분류기의 구현은 위에서 살펴본 간단한 RNN과 상당히 유사할 것이다.

In [4]:
class LSTMClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,(h,c) = self.rnn(x)
        return self.fc(h[-1])

이제 네트워크를 훈련시키겠습니다. LSTM 훈련도 상당히 느리고, 훈련 초기에는 정확도가 크게 높아 보이지 않을 수 있다는 점에 유의합니다. 또한 합리적인 학습 속도를 얻기 위해서는 `lr` 파라미터를 사용해야 할 수도 있습니다.

In [5]:
net = LSTMClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.259375
6400: acc=0.25859375
9600: acc=0.26177083333333334
12800: acc=0.2784375
16000: acc=0.313
19200: acc=0.3528645833333333
22400: acc=0.3965625
25600: acc=0.4385546875
28800: acc=0.4752777777777778
32000: acc=0.505375
35200: acc=0.5326704545454546
38400: acc=0.5557552083333334
41600: acc=0.5760817307692307
44800: acc=0.5954910714285714
48000: acc=0.6118333333333333
51200: acc=0.62681640625
54400: acc=0.6404779411764706
57600: acc=0.6520138888888889
60800: acc=0.662828947368421
64000: acc=0.673546875
67200: acc=0.6831547619047619
70400: acc=0.6917897727272727
73600: acc=0.6997146739130434
76800: acc=0.707109375
80000: acc=0.714075
83200: acc=0.7209134615384616
86400: acc=0.727037037037037
89600: acc=0.7326674107142858
92800: acc=0.7379633620689655
96000: acc=0.7433645833333333
99200: acc=0.7479032258064516
102400: acc=0.752119140625
105600: acc=0.7562405303030303
108800: acc=0.76015625
112000: acc=0.7641339285714286
115200: acc=0.7677777777777778
118400: acc=0.77112331081

(0.03487814127604167, 0.7728)

## Packed sequences

우리의 예시에서, 우리는 미니배치 안에 모든 시퀀스를 zero-vector로 패딩해야 했습니다. 약간의 메모리 낭비가 발생하지만, RNN을 사용하면 학습에 참여하지만 중요한 입력 정보를 전달하지 않는 패딩된 입력 항목에 대해 추가 RNN 셀을 만드는 것이 더 중요합니다. RNN을 실제 시퀀스 크기로만 훈련시키는 것이 훨씬 더 좋습니다.

이를 위해 PyTorch에는 특수한 형식의 패딩 시퀀스 스토리지가 도입되었습니다. 다음과 같은 입력 패딩 미니배치가 있다고 가정합니다:
```
[[1,2,3,4,5],
 [6,7,8,0,0],
 [9,0,0,0,0]]
```
여기서 0은 패딩된 값을 나타내며 입력 시퀀스의 실제 길이 벡터는`[5,3,1]`입니다.

패딩 된 시퀀스로 RNN을 효과적으로 훈련시키기 위해서 우리는 큰 미니배치 (`[1,6,9]`)로 첫 번째 그룹의 RNN 셀 훈련을 시작하고 세 번째 시퀀스의 처리를 종료하고 짧은 미니 배치 ( `[2,7]`,`[3,8]`) 입니다. 따라서 packed sequence는 하나의 벡터로 표현됩니다. 우리의 경우`[1,6,9,2,7,3,8,4,5]`와 길이 벡터 (`[5,3,1]`), 원래 패딩 된 미니 배치를 쉽게 재구성할 수 있습니다.

패킹 된 시퀀스를 생성하기 위해`torch.nn.utils.rnn.pack_padded_sequence` 함수를 사용할 수 있습니다. RNN, LSTM 및 GRU를 포함한 모든 순환 레이어는 packed sequence를 입력으로 지원하고 packed output을 생성하며, 이는 `torch.nn.utils.rnn.pad_packed_sequence` 를 사용하여 디코딩할 수 있습니다.

packed sequence를 생성하려면 길이 벡터를 네트워크에 전달해야하므로 미니배치를 준비하는 다른 함수가 필요합니다.

In [6]:
def pad_length(b):
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch and length sequence itself
    len_seq = list(map(len,v))
    l = max(len_seq)
    return ( # tuple of three tensors - labels, padded features, length sequence
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v]),
        torch.tensor(len_seq)
    )

train_loader_len = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=pad_length, shuffle=True)

실제 네트워크는 위의 `LSTMClassifier`와 매우 유사하지만 `forward` 패스는 패딩된 미니배치와 시퀀스 길이 벡터를 모두 입력으로 받습니다. 임베딩을 계산한 후 packed sequence를 계산하고 LSTM 레이어에 전달한 다음 결과를 다시 압축 해제(unpack)합니다.

> **참조**: 다음 계산에서 히든 레이어의 출력을 사용하기 때문에 실제로 압축 해제 된 결과 `x`를 사용하지 않습니다. 따라서 이 코드에서 압축 해제를 모두 제거할 수 있습니다. 여기에 배치하는 이유는 추가 계산에서 네트워크 출력을 사용해야 하는 경우이 코드를 쉽게 수정할 수 있기 때문입니다.

In [7]:
class LSTMPackClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x, lengths):
        batch_size = x.size(0)
        x = self.embedding(x)
        pad_x = torch.nn.utils.rnn.pack_padded_sequence(x,lengths,batch_first=True,enforce_sorted=False)
        pad_x,(h,c) = self.rnn(pad_x)
        x, _ = torch.nn.utils.rnn.pad_packed_sequence(pad_x,batch_first=True)
        return self.fc(h[-1])

이제 훈련을합시다.

In [8]:
net = LSTMPackClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch_emb(net,train_loader_len, lr=0.001,use_pack_sequence=True)


3200: acc=0.285625
6400: acc=0.33359375
9600: acc=0.3876041666666667
12800: acc=0.44078125
16000: acc=0.4825
19200: acc=0.5235416666666667
22400: acc=0.5559821428571429
25600: acc=0.58609375
28800: acc=0.6116666666666667
32000: acc=0.63340625
35200: acc=0.6525284090909091
38400: acc=0.668515625
41600: acc=0.6822596153846154
44800: acc=0.6948214285714286
48000: acc=0.7052708333333333
51200: acc=0.71521484375
54400: acc=0.7239889705882353
57600: acc=0.7315277777777778
60800: acc=0.7388486842105263
64000: acc=0.74571875
67200: acc=0.7518303571428572
70400: acc=0.7576988636363636
73600: acc=0.7628940217391305
76800: acc=0.7681510416666667
80000: acc=0.7728125
83200: acc=0.7772235576923077
86400: acc=0.7815393518518519
89600: acc=0.7857700892857142
92800: acc=0.7895043103448276
96000: acc=0.7930520833333333
99200: acc=0.7959072580645161
102400: acc=0.798994140625
105600: acc=0.802064393939394
108800: acc=0.8051378676470589
112000: acc=0.8077857142857143
115200: acc=0.8104600694444445
118400

(0.029785829671223958, 0.8138166666666666)

> **참조:** 훈련 함수에 전달하는 `use_pack_sequence` 파라미터를 알아차렸을 것입니다. 현재 `pack_padded_sequence` 함수는 길이 시퀀스 텐서가 CPU 장치에 있어야하므로 훈련 함수는 훈련시 길이 시퀀스 데이터를 GPU로 이동하지 않도록 해야합니다.[`torchnlp.py`](torchnlp.py) 파일에서 `train_emb` 함수 구현을 살펴볼 수 있습니다.

## Bidirectional and multilayer RNNs

이 예시에서 모든 순환 신경망은 시퀀스의 시작부터 끝까지 한 방향으로 작동했습니다. 우리가 말을 읽고 듣는 방식과 비슷하기 때문에 자연스럽게 보입니다. 그러나 많은 실사례에서의 입력 시퀀스에 임의로 액세스할 수 있으므로 양방향으로 반복 계산을 실행하는 것이 합리적일 수 있습니다. 이러한 네트워크는 **bidirectional** RNNs이라고하며, `bidirectional=True` 파라미터를 RNN / LSTM / GRU 생성자에 전달하여 만들 수 있습니다.

양방향 네트워크를 다룰 때는 각 방향에 하나씩 두 개의 은닉 상태 벡터(hidden state vectors)가 필요합니다. PyTorch는 이러한 벡터를 두 배 더 큰 크기의 벡터 하나로 인코딩합니다. 이는 일반적으로 결과 은닉 상태를 fully-connected 선형 레이어로 전달하고 레이어를 만들 때이 크기 증가를 고려하면 되기 때문에 매우 편리합니다.

단방향(one-directional) 또는 양방향(bidirectional)의 순환 네트워크는 시퀀스 내에서 특정 패턴을 캡처하여 상태 벡터에 저장하거나 출력으로 전달할 수 있습니다. 합성곱 네트워크(convolutional networks)와 마찬가지로 우리는 첫 번째 계층에서 추출한 단순한 패턴으로 빌드하여 더 높은 수준의 패턴을 캡처하기 위해 첫 번째 계층 위에 또 다른 반복 계층을 구축할 수 있다. 이것은 이전 레이어의 출력이 다음 레이어의 입력으로 전달되어 두 개 이상의 반복 네트워크로 구성된 **multi-layer RNN** 개념으로 이어집니다.

![Image showing a Multilayer long-short-term-memory- RNN](./images/multi-layer-lstm.jpg)

*Picture from [this wonderful post](https://towardsdatascience.com/from-a-lstm-cell-to-a-multilayer-lstm-network-with-pytorch-2899eb5696f3) by Fernando López*

PyTorch를 사용하면 여러 레이어의 반복을 자동으로 구축하기 위해 `num_layers` 파라미터를 RNN / LSTM / GRU 생성자에 전달하기만 하면  쉽게 구성할 수 있습니다. 또한 이것은 은닉 / 상태 벡터의 크기가 비례적으로 증가한다는 것을 의미하며, 반복 레이어의 출력을 처리할 때이를 고려해야합니다.

## RNNs for other tasks

이 단원에서는 RNN을 시퀀스 분류에 사용할 수 있지만 실제로는 텍스트 생성, 기계 번역 등과 같은 더 많은 작업을 처리 할 수 있음을 확인했습니다. 다음 단원에서 이러한 작업을 고려할 것입니다.